In [52]:
import pandas as pd 
import numpy as np
import re
import matplotlib.pyplot as plt
import seaborn as sns

from collections import Counter
from tqdm import tqdm

import time
import requests
import difflib
import operator
from difflib import SequenceMatcher
from fuzzywuzzy import fuzz 

In [6]:
import os
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
from pprint import pprint
from sklearn.model_selection import RandomizedSearchCV
from sklearn.svm import LinearSVC
from sklearn.metrics import confusion_matrix, accuracy_score

## **Load dataset**

In [23]:
df_skin = pd.read_csv('../Database/EWG/ewg_skin_products_transformed_8208.csv')
df_sun = pd.read_csv('../Database/EWG/ewg_sun_products_transformed_1565.csv')
df_makeup = pd.read_csv('../Database/EWG/ewg_makeup_products_transformed_10527.csv')

df_hair = pd.read_csv('../Database/EWG/ewg_hair_products_transformed_4908.csv')
df_or = pd.read_csv('../Database/EWG/ewg_oralcare_products_transformed_937.csv')
df_frag = pd.read_csv('../Database/EWG/ewg_fragrance_products_transformed_484.csv')

## **String cleaning**

In [55]:
def clean_string_name(string):
    text = convert_new_ing_list(string)
    text = text.replace(' Size: 4 oz. * USDA Certified Organic Ingredient ** None remains after saponifying oils into soap and glycerin', '')
    pattern = "[\|\*\_\'\{}&]".format('"')
    regex1 = re.compile('\[.*]')
    regex2 = re.compile('\[.*}')
    text = re.sub(pattern, "", text)
    text = re.sub(regex1, "", text)
    text = re.sub(regex2, "", text)
    text = text.strip('\x0c')
    text = text.strip(' ')
    text = text.upper()
    text = text.replace('ORGANIC', '')
    text = text.replace('SUSTAINABLE', '').replace('EXTRA VIRGIN', '').replace('SAPONIFIED', '')
    text = text.replace('AQUA (WATER)', 'WATER').replace('DISTILLED WATER', 'WATER').replace('WATER (AQUA)', 'WATER').replace('AQUA/WATER/EAU', 'WATER')
    text = text.replace('INACTIVE INGREDIENTS:', '')
    text = text.replace('ACTIVE INGREDIENTS:', '')

    return text

In [56]:
mismatch = ['CAVIAR WATER', 'STEM WATER']
def convert_new_ing_list(x):
    trimed = [name.strip(" '").strip("''") for name in x.strip('[]').split(',')]
    res = ''

    for i in range(len(trimed)):
        if i != (len(trimed) -1):
            if trimed[i] in mismatch:
                res = res + 'WATER' + ', '
            else:
                res = res + trimed[i] + ', '
        else: 
            if trimed[i] in mismatch:
                res = res + 'WATER' + ', '
            else:
                res = res + trimed[i]
    return res

In [57]:
def remove_water(string):
    water = ['WATER (AQUA)', 'AQUA', 'EAU', 'AQUA/WATER/EAU', 'AQUA / WATER / EAU', 
             'PURIFIED WATER', 'DISTILLED WATER', 'D.I. WATER', 'AQUA (WATER)']
    if string in water:
        string = 'WATER'
    text = string.strip()
    
    return text

In [58]:


class FuzzyDict(dict):
    "Provides a dictionary that performs fuzzy lookup"
    def __init__(self, items = None, cutoff = .6):
        """Construct a new FuzzyDict instance

        items is an dictionary to copy items from (optional)
        cutoff is the match ratio below which mathes should not be considered
        cutoff needs to be a float between 0 and 1 (where zero is no match
        and 1 is a perfect match)"""
        super(FuzzyDict, self).__init__()

        if items:
            self.update(items)
        self.cutoff =  cutoff

        # short wrapper around some super (dict) methods
        self._dict_contains = lambda key: \
            super(FuzzyDict,self).__contains__(key)

        self._dict_getitem = lambda key: \
            super(FuzzyDict,self).__getitem__(key)

    def _search(self, lookfor, stop_on_first = False):
        """Returns the value whose key best matches lookfor

        if stop_on_first is True then the method returns as soon
        as it finds the first item
        """

        # if the item is in the dictionary then just return it
        if self._dict_contains(lookfor):
            return True, lookfor, self._dict_getitem(lookfor), 1

        # set up the fuzzy matching tool
        #ratio_calc = difflib.SequenceMatcher()
        #ratio_calc.set_seq1(lookfor)

        # test each key in the dictionary
        best_ratio = 0
        best_match = None
        best_key = None
        for key in self:

            # if the current key is not a string
            # then we just skip it
            if not isinstance(key, str):
                continue

            # we get an error here if the item to look for is not a
            # string - if it cannot be fuzzy matched and we are here
            # this it is defintely not in the dictionary
            try:
            # calculate the match value
                ratio = fuzz.ratio(lookfor, key)/100
            except TypeError:
                break

            # if this is the best ratio so far - save it and the value
            if ratio > best_ratio:
                best_ratio = ratio
                best_key = key
                best_match = self._dict_getitem(key)

            if stop_on_first and ratio >= self.cutoff:
                break

        return (
            best_ratio >= self.cutoff,
            best_key,
            best_match,
            best_ratio)


    def __contains__(self, item):
        "Overides Dictionary __contains__ to use fuzzy matching"
        if self._search(item, True)[0]:
            return True
        else:
            return False

    def __getitem__(self, lookfor):
        "Overides Dictionary __getitem__ to use fuzzy matching"
        matched, key, item, ratio = self._search(lookfor)

        if not matched:
            raise KeyError(
                "'%s'. closest match: '%s' with ratio %.3f"%
                    (str(lookfor), str(key), ratio))

        return item

In [59]:
def compare_ingredient(x):
    x = clean_string_name(x)
    ing_list = [remove_water(name) for name in x.split(',')]
    match_dict = fuzzy_match_ingredients(ing_list, fd)
    res = [i for i in match_dict.values()]
#     print(sample)
#     print(res)
    
    return res

In [60]:
def fuzzy_match_ingredients(ing_list, fuzdict):
    match_dict = {}
    for ing in tqdm(ing_list):
        if ing in match_dict.keys():
            continue
        upper_ing = ing.upper()
        if fuzdict.__contains__(upper_ing):
            match_dict[ing] = fuzdict[upper_ing]
    
    return match_dict

In [61]:
df_skin = pd.read_csv('../Database/EWG/ewg_skin_products_transformed_8208.csv')
df_sun = pd.read_csv('../Database/EWG/ewg_sun_products_transformed_1565.csv')
df_makeup = pd.read_csv('../Database/EWG/ewg_makeup_products_transformed_10527.csv')

df_hair = pd.read_csv('../Database/EWG/ewg_hair_products_transformed_4908.csv')
df_or = pd.read_csv('../Database/EWG/ewg_oralcare_products_transformed_937.csv')
df_frag = pd.read_csv('../Database/EWG/ewg_fragrance_products_transformed_484.csv')

In [83]:
df_all = pd.concat([df_skin, df_sun, df_makeup, df_hair, df_or, df_frag])

In [85]:
df_all.duplicated().sum()

3365

In [87]:
df_all = df_all.drop_duplicates()

In [62]:
dfcosing = pd.read_csv('../Database/ingredient_cosing_37309.csv')

In [63]:
# dfi3 = pd.read_csv('Database/ingredient_inci_1570.csv')
ingnames_dict = {name.strip(): name.strip() for name in dfcosing['ingredient_name']}
# print(ingnames_dict)
fd = FuzzyDict(ingnames_dict, cutoff = .6)
print(len(fd))

29908


## **Testing**

In [78]:
df_test = df_skin.iloc[60:80, :]

In [88]:
df_all

,cat_name,subcat_name,product_url,product_brand,product_name,product_score,product_img,ingredient_list,new_ing_list
0,Skin,Bar Soap,https://www.ewg.org/skindeep/products/701958-Soap_for_Goodness_Sake_Handmade_Soap_Olive_Oil_I/,Soap for Goodness Sake,"Handmade Soap, Olive Oil I",verified,https://static.ewg.org/skindeep_images/7019/701958.jpg,"Saponified Organic Extra Virgin Olive Oil (Sodium Olivate), Distilled Water, Cinnamomum Zeylanicum (Cinnamon) Leaf Oil,","['STEM WATER', 'CINNAMOMUM ZEYLANICUM LEAF OIL']"
1,Skin,Bar Soap,https://www.ewg.org/skindeep/products/604412-Makes_3_Organics_Unscented_Super_Shea_Soap_Bar/,Makes 3 Organics,Unscented Super Shea Soap Bar,verified,https://static.ewg.org/skindeep_images/6044/604412.jpg,"Safflower* and/or Sunflower* Oils, Coconut* Oil, Palm Fruit* Oil, Shea Butter*, Water, Sodium Hydroxide**, Rosemary Extract. Size: 4 oz. * USDA Certified Organic Ingredient ** None remains after saponifying oils into soap and glycerin","['SAFFLOWER FLOWER', 'COCONUT ACID', 'PALM) FRUIT', 'SHEA BUTTERATE', 'WATER', 'SODIUM HYDROXIDE', 'ROSA EXTRACT']"
2,Skin,Bar Soap,https://www.ewg.org/skindeep/products/701974-Soap_for_Goodness_Sake_Handmade_Soap_Shea_and_Oats_/,Soap for Goodness Sake,"Handmade Soap, Shea and Oats",verified,https://static.ewg.org/skindeep_images/7019/701974.jpg,"Saponified Organic Sunflower Oil (Sodium Sunflower Seedate), Saponified Organic Palm Kernel Oil (Sodium Palm Kernelate), Saponified Organic Coconut Oil (Sodium Cocoate), Distilled Water, Butyrospermum Parkii (Shea Butter), Avena Sativa (Oat) Kernel Meal,","['STEM WATER', 'BUTYROSPERMUM PARKII BUTTER', 'AVENA SATIVA KERNEL MEAL']"
3,Skin,Bar Soap,https://www.ewg.org/skindeep/products/701929-Soap_for_Goodness_Sake_Handmade_Soap_Balancing_Act_/,Soap for Goodness Sake,"Handmade Soap, Balancing Act",verified,https://static.ewg.org/skindeep_images/7019/701929.jpg,"Saponified Organic Sunflower Oil (Sodium Sunflower Seedate), Saponified Organic Palm Kernel Oil (Sodium Palm Kernelate), Saponified Organic Coconut Oil (Sodium Cocoate), Distilled Water, Organic Simmondsia Chinensis (Jojoba), Organic Butyrospermum Parkii (Shea Butter), Avena Sativa (Oat) Kernel Meal, Natural Sea Salt (Sodium Chloride),","['STEM WATER', 'SIMMONDSIA CHINENSIS BUTTER', 'BUTYROSPERMUM PARKII BUTTER', 'AVENA SATIVA KERNEL MEAL', 'LAURALKONIUM CHLORIDE']"
4,Skin,Bar Soap,https://www.ewg.org/skindeep/products/701942-Soap_for_Goodness_Sake_Shampoo__Body_Bar_French_Lavender_/,Soap for Goodness Sake,"Shampoo & Body Bar, French Lavender",verified,https://static.ewg.org/skindeep_images/7019/701942.jpg,"Saponified Expeller Pressed Babassu Oil (Orbignya Oleifera Oil), Saponified Organic Sunflower Oil (Sodium Sunflower Seedate), Distilled Water, Organic Simmondsia Chinensis (Jojoba), Lavandula Angustifolia (Lavender) Flower Essential Oil, Organic Avena Sativa (Oat) Kernel Meal, Cinnamomum Zeylanicum (Cinnamon) Leaf Oil,","['STEM WATER', 'SIMMONDSIA CHINENSIS BUTTER', 'LAVANDULA ANGUSTIFOLIA FLOWER OIL', 'AVENA SATIVA KERNEL MEAL', 'CINNAMOMUM ZEYLANICUM LEAF OIL']"
...,...,...,...,...,...,...,...,...,...
478,Fragrance,Fragrance For Women,https://www.ewg.org/skindeep/products/877613-Calvin_Klein_Euphoria_Ladies_Parfum_Spray/,Calvin Klein,Euphoria Ladies Parfum Spray,10,https://static.ewg.org/skindeep/img/ewg_missing_product.png,"ALCOHOL DENAT., PARFUM/FRAGRANCE, AQUA/WATER/EAU, PROPYLENE GLYCOL, ETHYLHEXYL METHOXYCINNAMATE, LIMONENE, BENZYL SALICYLATE, BUTYL METHOXYDIBENZOYLMETHANE, ETHYLHEXYL SALICYLATE, LINALOOL, HEXYL CINNAMAL, CITRONELLOL, HYDROXYCITRONELLAL, BHT, COUMARIN, BUTYLPHENYL METHYLPROPIONAL, ALPHA-ISOMETHYL IONONE, CITRAL, GERANIOL, ISOEUGENOL, BENZYL BENZOATE, D&C GREEN NO.5 (CI 61570), D&C RED NO. 33 (CI 17200), FD&C YELLOW NO. 5 (CI 19140), EXT. D&C VIOLET NO. 2 (C1 60730).","['ALCOHOL DENAT.', 'APAFLURANE', 'PROPYLENE GLYCOL', 'ETHYLHEXYL METHOXYCINNAMATE', 'LIMONENE', 'BENZYL SALICYLATE', 'BUTYL METHOXYDIBENZOYLMETHANE', 'ETHYLHEXYL SALICYLATE', 'LINALOOL

In [80]:
# dfi3 = pd.read_csv('Database/ingredient_inci_1570.csv')
ingnames_dict = {name.strip(): name.strip() for name in dfcosing['ingredient_name']}
# print(ingnames_dict)
fd = FuzzyDict(ingnames_dict, cutoff = .6)
print(len(fd))

29908


In [ ]:
df_all['new_ing_list'] = df_all['ingredient_list'].apply(compare_ingredient)

100%|██████████| 8/8 [00:00<00:00,  9.48it/s]


100%|██████████| 11/11 [00:00<00:00, 12.77it/s]


100%|██████████| 11/11 [00:01<00:00,  6.79it/s]


100%|██████████| 23/23 [00:02<00:00,  8.86it/s]


100%|██████████| 5/5 [00:00<00:00, 45100.04it/s]


...

100%|██████████| 32/32 [00:02<00:00, 14.77it/s]


100%|██████████| 9/9 [00:02<00:00,  3.94it/s]


100%|██████████| 7/7 [00:01<00:00,  3.75it/s]


100%|██████████| 14/14 [00:02<00:00,  5.72it/s]


 71%|███████   | 12/17 [00:06<00:02,  1.67it/s]

In [ ]:
df_all.to_csv('ewg_products_transformed_23260_20201027.csv', index=False)

In [8]:
df_all = pd.concat([df_skin, df_sun, df_makeup, df_hair, df_or, df_frag])

In [14]:
pd.set_option('display.max_colwidth', -1)
df_all.iloc[300:360, :][['ingredient_list', 'new_ing_list']]

2020-10-27 00:58:08,671 [2410] WARNING  py.warnings:110: [JupyterRequire] /Users/macbookpro/miniconda3/envs/cs_ftmle/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  """Entry point for launching an IPython kernel.



,ingredient_list,new_ing_list
300,"SAPONIFIED PALM OIL, VEGETABLE GLYCERIN, SODIUM GLUCONATE, COCONUT OIL, FRAGRANCE BLEND WITH PLANT EXTRACTS AND/OR ESSENTIAL OILS, OLIVE OIL, CHARCOAL POWDER, SODIUM CHLORIDE, IRON OXIDES.","['ISOMERIZED PALM OIL', 'VEGETABLE GLYCERIDES PHOSPHATE', 'SODIUM GLUCONATE', 'COCONUT ACID', 'OLIVE OIL', 'CHARCOAL POWDER', 'SODIUM CHLORIDE', 'IRON OXIDES']"
301,"SODIUM COCOATE, SODIUM OLIVATE, GOAT MILK, SODIUM SHEA BUTTERATE, SODIUM COCOA BUTTERATE, SODIUM RICINOLEATE, FRAGRANCE AND/OR ESSENTIAL OIL, KAOLIN CLAY","['SODIUM COCOATE', 'SODIUM OLIVATE', 'GOAT MILK', 'SODIUM SHEA BUTTERATE', 'SODIUM COCOA BUTTERATE', 'SODIUM RICINOLEATE', 'KAOLIN']"
302,"Vegetable Oil Blend (Coconut Oil, Olive Oil, Organic Palm Oil, Shea Butter), Water, Sodium Hydroxide, Fragrance Oil Blend, Lavender Essential Oil), Ultramarines, Mica, Titanium Dioxide, Lavender Buds.","['HYDROGENATED COCONUT OIL', 'OLIVE OIL', 'ISOMERIZED PALM OIL', 'SHEA BUTTERATE', 'WATER', 'SODIUM HYDROXIDE', 'ALTERNANTHERA SESSILIS OIL', 'ULTRAMARINES', 'MICA', 'TITANIUM DIOXIDE', 'LAVENDER']"
303,"SODIUM PALMATE, SODIUM COCOATE, SODIUM PALM KERNELATE, PURIFIED WATER, CAPRAE LAC (GOAT MILK) POWDER, PRUNNUS AMYGDALUS DULCIS (SWEET ALMOND) OIL, GLYCERIN, SODIUM CHLORIDE, TOCOPHEROL ACETATE, TETRA SODIUM EDTA, FRAGRANCE BACKHOUSIA CITRIODORA (LEMON MYRTLE) LEAF OIL.","['SODIUM PALMATE', 'SODIUM COCOATE', 'SODIUM PALM KERNELATE', 'PEAT WATER', 'CHARCOAL POWDER', 'PRUNUS AMYGDALUS DULCIS SEED', 'GLYCERIN', 'SODIUM CHLORIDE', 'TOCOPHERYL ACETATE', 'TETRASODIUM EDTA', 'BACKHOUSIA CITRIODORA LEAF OIL']"
304,"SODIUM PALMATE (SAPONIFIED MASS BALANCE SUSTAINABLE PALM OIL), SODIUM COCOATE (SAPONIFIED COCONUT OIL), (WATER), GLYCERIN (RETAINED VEGETABLE GLYCERIN), PARFUM (COCONUT AND PAPAYA FRAGRANCE), COCOS NUCIFERA (COCONUT) OIL, ORGANIC BUTYROSPERMUM PARKII (SHEA) BUTTER1, VANILLA PLANIFOLIA (VANILLA BEANS), SODIUM CHLORIDE, CARICA PAPAYA (PAPAIN) FRUIT EXTRACT, COCOS NUCIFERA FRUIT POWDER (TOASTED COCONUT), ORGANIC TOCOPHEROL (VITAMIN E), SODIUM GLUCONATE, ROSMARINUS OFFICINALIS (ROSEMARY) LEAF EXTRACT","['SULFATED COCONUT OIL', 'WATER', 'HYDROGENATED VEGETABLE GLYCERIDE', 'COCOS NUCIFERA OIL', 'BUTYROSPERMUM PARKII BUTTER', 'VANILLA PLANIFOLIA LEAF CELL EXTRACT', 'SODIUM CHLORIDE', 'CARICA PAPAYA FRUIT EXTRACT', 'COCOS NUCIFERA FRUIT POWDER', 'SODIUM GLUCONATE', 'ROSMARINUS OFFICINALIS LEAF EXTRACT']"
305,"SODIUM PALMATE (DERIVED FROM RAINFOREST ALLIANCE CERTIFIED PALM OIL), SODIUM COCOATE (OR) SODIUM PALM KERNELATE, WATER (AQUA), GLYCERIN, COCONUT ACID, BUTYROSPERMUM PARKII BUTTER (ORGANIC, FAIR-TRADE, HAND-CRAFTED SHEA BUTTER), FRAGRANCE (PARFUM), SODIUM CHLORIDE, GALACTOARABINAN, SODIUM GLUCONATE, TETRASODIUM GLUTAMATE DIACETATE, CLADONIA STELLARIS EXTRACT (ALPINE CARIBOU MOSS), KAOLIN, SIMMONDSIA CHINENSIS (JOJOBA) SEED OIL, JUNIPERUS COMMUNIS FRUIT.","['SODIUM PALM KERNELATE', 'MARIS AQUA', 'GLYCERIN', 'COCONUT ACID', 'BUTYROSPERMUM PARKII BUTTER EXTRACT', 'TARTRATE', 'HYDROGENATED SHEA BUTTER', 'SODIUM CHLORIDE', 'GALACTOARABINAN', 'SODIUM GLUCONATE', 'TETRASODIUM GLUTAMATE DIACETATE', 'KAOLIN', 'SIMMONDSIA CHINENSIS SEED OIL', 'JUNIPERUS COMMUNIS FRUIT']"
306,"Sodium Olivate, Sodium Cocoate, Sodium Stearate, Aqua (Water), Citrus Aurantifolia Oil (Lime Oil), Glycerin, Sodium Chloride, CI 77019 (Mica), Zingiber Officinale Root Oil (Ginger Oil), Tetrasodium Glutamate Diacetate, *Citral, * d-Limonene.","['SODIUM OLIVATE', 'SODIUM COCOATE', 'SODIUM STEARATE', 'CAVIAR WATER', 'CITRUS AURANTIFOLIA LEAF OIL', 'GLYCERIN', 'SODIUM CHLORIDE', 'CI 77015', 'ZINGIBER OFFICINALE ROOT OIL', 'TETRASODIUM GLUTAMATE DIACETATE', 'CITRAL', 'LIMONENE']"
307,"SODIUM PALM KERNELATE, WATER, SODIUM PALMATE, POTASSIUM PALM KERNELATE, PALM KERNEL ACID, FRAGRANCE/PARFUM, GLYCERIN, POTASSIUM PALMATE, SODIUM GLUCONATE, TITANIUM DIOXIDE (CI 77891), COCOS NUCIFERA (COCONUT) OIL, BUTYROSPERMUM PARKII (SHEA) BUTTER, SODIUM CHLORIDE, PALM ACID, LIMONENE, BENZYL SALICYLATE, LINALOOL, AL